In [1]:
import os
import pandas as pd

In [2]:
article_df = pd.read_parquet("gs://scraped-news-article-data-null/2023-coref.parquet")
article_df.head()

,source,id,category,title,published,body,summary,summary_type
0,reuters,29667,Europe,European shares rise on upbeat cues from Powel...,2023-02-08T17:11:00,Feb 8 (Reuters) - European shares rose on Wedn...,* \n* STOXX 600 pulls back from 9-mth high to ...,BULLETS
1,reuters,79619,China,China Evergrande debt restructuring incentive ...,2023-04-27T06:05:00,"HONG KONG, April 27 (Reuters) - Embattled prop...",,NULL
2,reuters,48010,European Markets,European stocks rally as banking worries fade,2023-03-30T16:27:00,March 30 (Reuters) - European stocks rose to n...,* \n* H&M posts surprise profit in Dec-Feb\n* ...,BULLETS
3,reuters,67859,Aerospace & Defense,China gears up to compete with SpaceX's Starli...,2023-03-02T10:29:00,"BEIJING, March 2 (Reuters) - China's military-...",,NULL
4,reuters,109054,World,Russia's war on Ukraine latest: Moscow denies ...,2023-05-12T05:13:00,May 11 (Reuters) - Russia's defence ministry o...,,NULL


In [6]:
len(article_df.index)

61921

In [3]:
import json

In [4]:
result = []
counter = 1

for index, r, *l in article_df.iterrows():
    if r.summary_type == "NULL":
        continue
    if r.summary_type == "BULLETS":
        human = "summarize the keypoints from following article in bullet points. Ensure that the keypoints read well.\n"
    if r.summary_type == "PLAIN":
        human = "summarize the following article. Ensure that the keypoints read well.\n"
        
    human = human + r.body
    gpt = str(r.summary)
    
    conversation = [
        {"from": "human", "value": human},
        {"from": "gpt", "value": gpt}
    ]
    
    output_dict = {
        "id": counter,
        "conversations": conversation
    }
    result.append(output_dict)
    counter = counter + 1


In [5]:
with open("/home/jupyter/fine_tune.json", "w") as fp:
    json.dump(fp=fp, obj=result)